# 3. Espacialização de Dados Locais para Pontos do CMIP6

```python
Esse caderno tem como objetivo a obtenção da precipitação de dados locais 
para os pontos definidos nos GCMs do CMIP6 a partir de interpolação espacial.
```

In [4]:
import os
import numpy as np
import pandas as pd

from IPython.display import clear_output
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor
)

from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import (
    Conv1D,
    MaxPooling1D,
    Flatten,
    Dense,
    Dropout
)

## 3.1 Configurações

In [5]:
# Definição de se vai ocorrer ou não a geração de bases de dados
databases_generate = True

# Definição de se vai ocorrer ou não o a geração e o uso do método IDW
idw_method = True
idw_generate = False

# Tipo de base de dados local utilizada ('sum' ou 'max')
database_type = 'sum'

## 3.2 Funções

### 3.1.1. Função para Limeza de Terminal e Células

In [6]:
def clear():
    '''
    Função para limpar terminal ou célula
    '''

    # Limpando terminal
    os.system('cls')

    # Limpando célula
    clear_output(wait=True)

### 3.1.2. Função que Adiciona coluna IDW à Base de Dados

In [7]:
def porcentagem_em_barra(valor_atual: int,
                         valor_total: int) -> str:
    """
    Gerador de barra de porcentagem a partir de valor atual e total.
    """

    porcentagem = 100 * (valor_atual / valor_total)

    completo   = '-' * (int(porcentagem))
    incompleto = '_' * (100 - int(porcentagem))

    situacao = f'[{completo}{incompleto}] {porcentagem:.2f}% ({valor_atual} de {valor_total})'

    return situacao

def vizinhos_proximos(lat: float,
                      lon: float,
                      lat_lon: list[str],
                      n_vizinhos: int = 5):
    """
    Gerador de lista de pontos vizinhos próximos de determinado ponto.

    Args:
        lat (float): Latitude do ponto principal;
        lon (float): Logitude do ponto principal;
        lat_lon (list[str]): Lista com latitudes e longitudes de pontos próximos ao ponto principal;
        n_vizinhos (int, optional): Números de pontos vizinhos ao ponto principal a se estimar.

    Returns:
        list: Lista de n pontos mais próximos ao ponto principal.
    """

    # Lista para armazenar tuplas (string_original, distancia)
    distancias = []

    for ponto_str in lat_lon:
        lat_p, lon_p = map(float, ponto_str.split(";"))
        dist = np.linalg.norm(np.array([lat, lon]) - np.array([lat_p, lon_p]))
        distancias.append((ponto_str, dist))

    # Ordena pela menor distância
    distancias_ordenadas = sorted(distancias, key=lambda x: x[1])

    # Pega os n vizinhos mais próximos (ignorando o primeiro se for o próprio ponto)
    vizinhos = [p[0] for p in distancias_ordenadas if p[1] != 0][:n_vizinhos]

    return vizinhos

def interpolacao_por_idw(df: pd.DataFrame,
                         var_de_predicao: str,
                         var_de_anos: str,
                         var_de_meses: str,
                         var_de_pontos: str,
                         pontos: str = 'all',
                         progresso: bool = True,
                         constante: int = 2,
                         n_vizinhos: int = 5) -> list:
    '''
    Interpola dados de séries temporais a partir do método IDW, e cria uma coluna para tal
    '''

    # Definindo nova coluna para o IDW
    df.loc[:, 'IDW'] = np.nan

    # Obtendo pontos únicos
    if pontos == 'all':
        pontos_unicos = df[var_de_pontos].unique()
    else:
        pontos_unicos = [pontos]

    # Varendo pontos únicos
    for i, ponto in enumerate(pontos_unicos):

        if progresso == True:

            # Ponto em cálculo
            print(porcentagem_em_barra(i+1, len(pontos_unicos)))

        lat, lon = map(float, ponto.split(";"))

        # Obtendo anos únicos
        anos_unicos = df[df[var_de_pontos] == ponto][var_de_anos].unique()

        # Varendo anos únicos dos pontos únicos
        for ano in anos_unicos:

            # Obtendo meses únicos
            meses_unicos = df[(df[var_de_pontos] == ponto) &
                              (df[var_de_anos] == ano)][var_de_meses].unique()

            # Varrendo meses únicos dos anos únicos dos pontos únicos
            for mes in meses_unicos:

                # Filtrando pontos unicos que possuem mesmo mes e ano que o ponto em varredura
                pontos_unicos_filtrados = df[(df[var_de_anos] == ano) &
                                             (df[var_de_meses] == mes)][var_de_pontos].unique()

                # Obtendo pontos vizinhos mais próximos do ponto em varredura
                vizinhos = vizinhos_proximos(lat, lon, pontos_unicos_filtrados, n_vizinhos)

                # Definindo variáveis para calcular IDW
                dividendo = divisor = 0

                for ponto_vizinho in vizinhos:

                    lat_vizinha, lon_vizinha = map(float, ponto_vizinho.split(";"))

                    distancia = ((lat - lat_vizinha)**2 + (lon - lon_vizinha)**2)**(1/2)

                    variavel = df.loc[(df[var_de_pontos] == ponto_vizinho) &
                                      (df[var_de_anos] == ano) &
                                      (df[var_de_meses] == mes), var_de_predicao]

                    if not variavel.empty:
                        valor = variavel.iloc[0]
                    else:
                        continue

                    dividendo += valor / (distancia**constante)

                    divisor += 1 / (distancia**constante)

                if divisor == 0:
                    idw = np.nan
                else:
                    idw = dividendo / divisor

                idx = df.loc[(df[var_de_pontos] == ponto) &
                             (df[var_de_anos] == ano) &
                             (df[var_de_meses] == mes), 'IDW'].index[0]

                df.at[idx, 'IDW'] = idw

        clear()

    df['IDW'] = df['IDW'].fillna(df['IDW'].mean())

    return df

### 3.1.3. Interpolação a partir de Modelos de Machine Learning

In [8]:
def interpolacao_por_ml(df: pd.DataFrame,
                        col_de_treino: list[str],
                        var_de_predicao: str,
                        var_de_pontos: str,
                        n_de_teste: int):

    # Obtendo pontos únicos
    pontos_unicos = df[var_de_pontos].unique()

    # Definição dos modelos
    models = [

        ("ExtraTrees", ExtraTreesRegressor(
            n_estimators=200,
            max_depth=20,
            max_features=2,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=7
        )),

        ("RandomForest", RandomForestRegressor(
            n_estimators=300,
            max_depth=25,
            max_features=2,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=7
        )),

        ("KNeighbors", KNeighborsRegressor(
            n_neighbors=7,
            weights='distance',
            algorithm='auto'
        )),

        ("GradientBoosting", GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.8,
            random_state=7
        )),

        # ("LinearRegression", LinearRegression(
        #     fit_intercept=True,
        #     positive=False
        # ))

    ]

    # Defininção de lista de métricas
    metrics = []
    for i in range(len(models)):
        metrics.append([[], []])

    for i in range(n_de_teste):

        # Embaralha os pontos únicos
        np.random.shuffle(pontos_unicos)

        # Dividindo em 70% treino e 30% teste
        split_idx = int(len(pontos_unicos) * 0.8)
        pontos_treino = set(pontos_unicos[:split_idx])
        pontos_teste = set(pontos_unicos[split_idx:])

        # Criando DataFrames de treino e teste
        df_treino = df[df[var_de_pontos].isin(pontos_treino)].copy()
        df_teste = df[df[var_de_pontos].isin(pontos_teste)].copy()

        # Definindo features (X) e variável alvo (y)
        X_train = df_treino[col_de_treino]
        y_train = df_treino[var_de_predicao]

        X_test = df_teste[col_de_treino]
        y_test = df_teste[var_de_predicao]

        # Treinar e avaliar cada modelo
        for j in range(len(models)):
            models[j][1].fit(X_train, y_train)                                 # Treinamento
            y_pred = models[j][1].predict(X_test)                              # Previsão
            metrics[j][0].append(r2_score(y_test, y_pred))                     # R²
            metrics[j][1].append(np.sqrt(mean_squared_error(y_test, y_pred)))  # RMSE

    # Verificando melhor Modelo a partir de r2
    best_model = (0, '', '')

    print('Verificação de Modelos:\n')

    for i in range(len(metrics)):

        r2, rmse = np.mean(metrics[i][0]), np.mean(metrics[i][1])

        if best_model[0] < r2:
            best_model = r2, models[i][0], models[i][1]

        print(f'Modelo: {models[i][0][:3]} \t R²: {r2:.4f} \t RMSE: {rmse:.4f}')

    print(f'\nO melhor modelo de ML para a base de dados é: {best_model[1]}.')

    return best_model[2]

### 3.1.4. Interpolação a partir de Modelo de Deep Learning

In [9]:
def interpolacao_por_cnn(df: pd.DataFrame,
                         col_de_treino: list[str],
                         var_de_predicao: str,
                         var_de_pontos: str,
                         n_de_teste: int):

    # Pontos únicos
    pontos_unicos = df[var_de_pontos].unique()

    # Listas para métricas
    r2_list = []
    rmse_list = []

    for i in range(n_de_teste):

        np.random.shuffle(pontos_unicos)
        split_idx = int(len(pontos_unicos) * 0.8)

        pontos_treino = set(pontos_unicos[:split_idx])
        pontos_teste = set(pontos_unicos[split_idx:])

        df_treino = df[df[var_de_pontos].isin(pontos_treino)].copy()
        df_teste = df[df[var_de_pontos].isin(pontos_teste)].copy()

        # Definindo X e y
        X_train = df_treino[col_de_treino].values
        y_train = df_treino[var_de_predicao].values

        X_test = df_teste[col_de_treino].values
        y_test = df_teste[var_de_predicao].values

        # Redimensionar para 3D: (samples, timesteps=1, features)
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Limpar sessão anterior (importante em loops com Keras)
        clear_session()

        # Criando modelo CNN
        cnn = Sequential([
            Input(shape=(1, X_train.shape[2])),
            Conv1D(64, kernel_size=3, padding='same', activation='relu'),
            MaxPooling1D(1),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dense(1)
        ])

        cnn.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])

        # Treinamento
        cnn.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=0)

        # Previsão e avaliação
        y_pred = cnn.predict(X_test).flatten()

        r2_list.append(r2_score(y_test, y_pred))
        rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    # Resultados finais
    print(f"\nCNN \t Média R²: {np.mean(r2_list):.4f} \t Média RMSE: {np.mean(rmse_list):.4f}")

    return cnn

### 3.1.5. Preenchimento para Serie Temporal Completa

In [10]:
def complete_temporal_series(df: pd.DataFrame,
                             lat_col: str,
                             lon_col: str,
                             anos: list) -> pd.DataFrame:
    '''
    Preenchimento de datas faltantes em séries temporais
    '''

    # Obtendo pontos de lat lon únicos
    df_pontos_unicos = df[[lat_col, lon_col]].drop_duplicates().reset_index().drop(columns=["index"])

    # Obtendo meses e anos da base de dados
    df_anos_meses = pd.DataFrame([(ano, mes) for ano in anos for mes in range(1, 13)], columns=["ano", "mes"])

    # Fazendo produto cartesiano entre pontos e anos/meses
    df_pontos_unicos['key'] = 1
    df_anos_meses['key'] = 1

    # Unindo pontos e anos/meses
    df_expandido = pd.merge(df_pontos_unicos, df_anos_meses, on='key').drop(columns='key')

    # Cria um DataFrame vazio para armazenar os dados expandidos
    return df_expandido

## 3.2. Coluna IDW para dados da AESA

### 3.2.1. Configurando Bases de Dados

In [11]:
# Base de dados local de acumulados
df_aesa = pd.read_csv(
    f"../datas/interim/2.3.1_aesa_database_create/aesa_1994-2023_mon_{database_type}.csv"
)

# Caso não tenha a coluna IDW nas "colunas_de_interesse", calcula-se o IDW
if idw_method == True:

    if idw_generate == True:  # 115m 33.8s

        # Local
        df_aesa = interpolacao_por_idw(df_aesa, "pr_local", "ano", "mes", "pnt")
        df_aesa.to_csv(f'../datas/interim/3.2.1_aesa_with_idw/aesa_1994-2023_mon_{database_type}_idw.csv')

    else:

        # Local
        df_aesa = pd.read_csv(f'../datas/interim/3.2.1_aesa_with_idw/aesa_1994-2023_mon_{database_type}_idw.csv')

# Visualizando Bases de Dados Locais
print('- Informações Locais:')
print(df_aesa.info())
# grafico_de_pontos(df_aesa, "lat", "lon")

- Informações Locais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87120 entries, 0 to 87119
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  87120 non-null  int64  
 1   Unnamed: 0    87120 non-null  int64  
 2   lat           87120 non-null  float64
 3   lon           87120 non-null  float64
 4   alt           87120 non-null  float64
 5   bac           87120 non-null  object 
 6   ano           87120 non-null  int64  
 7   mes           87120 non-null  int64  
 8   pr_local      87120 non-null  float64
 9   pnt           87120 non-null  object 
 10  IDW           87120 non-null  float64
dtypes: float64(5), int64(4), object(2)
memory usage: 7.3+ MB
None


## 3.3. Base de Dados Local para Predição

```python
O processo consiste em criar uma base de dados com os mesmos 
pontos da base de dados do GCM, para que assim se possa realizar 
a interpolação da precipitação local para esses pontos.
```

### 3.3.1. Configurando Base de Dados Local para Predição

In [12]:
# Geração ou abertura de base de dados gerada
if databases_generate == True:

    # Definindo base de dados de GCM
    df_cnrm_cm6_1hr = pd.read_csv(
        f"../datas/interim/1.3.2_cmip6_database_create/pr_day_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gr_19940101-21001231_{database_type}.csv"
    )

    # Definindo base de dados
    df_aesa_to_cnrm_cm6_1hr = complete_temporal_series(df_cnrm_cm6_1hr, 'lat', 'lon', [i for i in range(1994, 2024)])

    # Cria uma coluna "pnt" que combina latitude e longitude como string separada por ponto e vírgula
    df_aesa_to_cnrm_cm6_1hr['pnt'] = df_aesa_to_cnrm_cm6_1hr["lat"].astype(str) + ";" + df_aesa_to_cnrm_cm6_1hr["lon"].astype(str)

    # Exportando base de dados gerada
    df_aesa_to_cnrm_cm6_1hr.to_csv(f'../datas/interim/3.3.1_aesa_in_cmip6_points/aesa_to_cnrm_cm6_1hr_{database_type}.csv')

else:

    # Abrindo base de dados
    df_aesa_to_cnrm_cm6_1hr = pd.read_csv(f'../datas/interim/3.3.1_aesa_in_cmip6_points/aesa_to_cnrm_cm6_1hr_{database_type}.csv')

# Informaões da base de dados gerada
df_aesa_to_cnrm_cm6_1hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     18000 non-null  float64
 1   lon     18000 non-null  float64
 2   ano     18000 non-null  int64  
 3   mes     18000 non-null  int64  
 4   pnt     18000 non-null  object 
dtypes: float64(2), int64(2), object(1)
memory usage: 703.3+ KB


### 3.3.2. Configurando Base de Dados Local para Predição com IDW

In [13]:
# Caso não tenha a coluna IDW nas "colunas_de_interesse", calcula-se o IDW
if idw_method == True:

    if idw_generate == True:  # 25m 35.1s

        # Adicionando coluna IDW à base de dados

        df_temp = pd.DataFrame()

        for _, row in df_aesa_to_cnrm_cm6_1hr.iterrows():

            row_temp = interpolacao_por_idw(pd.concat([df_aesa, pd.DataFrame([row])], ignore_index=True), "pr_local", "ano", "mes", "pnt", row['pnt'], False)

            df_temp = pd.concat([df_temp, row_temp[row_temp['pnt'] == row['pnt']]], ignore_index=True)

            print(porcentagem_em_barra(_+1, len(df_aesa_to_cnrm_cm6_1hr)))

        df_temp.to_csv(f'../datas/interim/3.3.2_aesa_in_cmip6_points_with_idw/aesa_to_cnrm_cm6_1hr_{database_type}_idw.csv')

        df_aesa_to_cnrm_cm6_1hr = df_temp.copy()

    else:

        # Abrindo base de dados
        df_aesa_to_cnrm_cm6_1hr = pd.read_csv(f'../datas/interim/3.3.2_aesa_in_cmip6_points_with_idw/aesa_to_cnrm_cm6_1hr_{database_type}_idw.csv')

# Informaões da base de dados gerada
df_aesa_to_cnrm_cm6_1hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  18000 non-null  int64  
 1   Unnamed: 0    0 non-null      float64
 2   lat           18000 non-null  float64
 3   lon           18000 non-null  float64
 4   alt           0 non-null      float64
 5   bac           0 non-null      float64
 6   ano           18000 non-null  int64  
 7   mes           18000 non-null  int64  
 8   pr_local      0 non-null      float64
 9   pnt           18000 non-null  object 
 10  IDW           18000 non-null  float64
dtypes: float64(7), int64(3), object(1)
memory usage: 1.5+ MB


#### 3.3.3. Configurando Modelo Preditivo de Interpolação

In [14]:
# Definindo colunas para treino
if idw_method == True:
    columns_X = ["lat", "lon", "ano", "mes", "IDW"]
else:
    columns_X = ["lat", "lon", "ano", "mes"]

# Escolhendo melhor modelo preditivo de ML
model = interpolacao_por_ml(df_aesa, columns_X, "pr_local", "pnt", 5)

# # Escolhendo melhor modelo preditivo de DL
# model = interpolacao_por_cnn(df_aesa, columns_X, "pr_local", "pnt", 5)

# Definindo features (X) e variável alvo (y)
X = df_aesa[columns_X].copy()
y = df_aesa["pr_local"].copy()

# Treinamento
model.fit(X, y)

Verificação de Modelos:

Modelo: Ext 	 R²: 0.7826 	 RMSE: 37.5774
Modelo: Ran 	 R²: 0.7721 	 RMSE: 38.4632
Modelo: KNe 	 R²: 0.7517 	 RMSE: 40.1451
Modelo: Gra 	 R²: 0.7712 	 RMSE: 38.5478

O melhor modelo de ML para a base de dados é: ExtraTrees.


ExtraTreesRegressor(max_depth=20, max_features=2, n_estimators=200,
                    random_state=7)

In [15]:
# Previsão dos valores de 'pr_local' com o modelo treinado
pr_local = model.predict(df_aesa_to_cnrm_cm6_1hr[columns_X])

# Colunas do Modelo
columns = ['lat', 'lon', 'ano', 'mes', 'pr_local']

# Adicionando a nova coluna 'pr_local' ao DataFrame
df_aesa_to_cnrm_cm6_1hr['pr_local'] = pr_local

# Salvando dados preditos
df_aesa_to_cnrm_cm6_1hr[columns].to_csv(f'../datas/interim/3.3.3_aesa_interpolated_to_cmip6/aesa_to_cnrm_cm6_1hr_{database_type}_interpolated.csv')

# Exibindo as primeiras linhas para verificar
df_aesa_to_cnrm_cm6_1hr[columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lat       18000 non-null  float64
 1   lon       18000 non-null  float64
 2   ano       18000 non-null  int64  
 3   mes       18000 non-null  int64  
 4   pr_local  18000 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 703.3 KB


## 3.3. Base de Dados GCM para Predição (Inutilizado)

```shell
O processo consiste em criar uma base de dados com os mesmos 
pontos da base de dados locais, para que assim se possa realizar 
a interpolação da precipitação do GCM para esses pontos.
```

### 3.3.1. Configurando Base de Dados do GCM para Predição

```python
# Geração ou abertura de base de dados gerada
if databases_generate == True:

    # Definindo base de dados
    df_cnrm_cm6_1hr_to_aesa = complete_temporal_series(df_aesa, 'lat', 'lon', [i for i in range(1994, 2024)])

    # Cria uma coluna "pnt" que combina latitude e longitude como string separada por ponto e vírgula
    df_cnrm_cm6_1hr_to_aesa['pnt'] = df_cnrm_cm6_1hr_to_aesa["lat"].astype(str) + ";" + df_cnrm_cm6_1hr_to_aesa["lon"].astype(str)

    # Exportando base de dados gerada
    df_cnrm_cm6_1hr_to_aesa.to_csv(f'cnrm_cm6_1hr_to_aesa_{database_type}.csv')

else:

    # Abrindo base de dados
    df_cnrm_cm6_1hr_to_aesa = pd.read_csv(f'cnrm_cm6_1hr_to_aesa_{database_type}.csv')

# Informaões da base de dados gerada
df_cnrm_cm6_1hr_to_aesa.info()
```

#### 3.3.2. Configurando Base de Dados de GCM para Predição com IDW

```python
# Caso não tenha a coluna IDW nas "colunas_de_interesse", calcula-se o IDW
if idw_method == True:

    if idw_generate == True:  # 97m 54.1s
        
        # Adicionando coluna IDW à base de dados

        df_temp = pd.DataFrame()

        for _, row in df_cnrm_cm6_1hr_to_aesa.iterrows():   

            row_temp = interpolacao_por_idw(pd.concat([df_cnrm_cm6_1hr, pd.DataFrame([row])], ignore_index=True), "pr", "ano", "mes", "pnt", row['pnt'], False)
            
            df_temp = pd.concat([df_temp, row_temp[row_temp['pnt'] == row['pnt']]], ignore_index=True)

            print(porcentagem_em_barra(_+1, len(df_cnrm_cm6_1hr_to_aesa)))

        df_temp.to_csv(f'cnrm_cm6_1hr_to_aesa_{database_type}_idw.csv')

        df_cnrm_cm6_1hr_to_aesa = df_temp.copy()

    else:
        
        # Abrindo base de dados
        df_cnrm_cm6_1hr_to_aesa = pd.read_csv(f'cnrm_cm6_1hr_to_aesa_{database_type}_idw.csv')

# Informaões da base de dados gerada
df_cnrm_cm6_1hr_to_aesa.info()
```

#### 3.3.3. Configurando Modelo Preditivo de Interpolação

```python
# Definindo colunas para treino
if idw_method == True:
    columns_X = ["lat", "lon", "ano", "mes", "IDW"]
else:
    columns_X = ["lat", "lon", "ano", "mes"]

# Escolhendo melhor modelo preditivo
model = interpolacao_por_ml(df_cnrm_cm6_1hr, columns_X, "pr", "pnt", 5)

# Definindo features (X) e variável alvo (y)
X = df_cnrm_cm6_1hr[columns_X].copy()
y = df_cnrm_cm6_1hr["pr"].copy()

# Treinamento
model.fit(X, y)
```

```python
# Previsão dos valores de 'pr' com o modelo treinado
pr = model.predict(df_cnrm_cm6_1hr_to_aesa[columns_X])

# Colunas do Modelo
columns = ['lat', 'lon', 'ano', 'mes', 'pr']

# Adicionando a nova coluna 'pr' ao DataFrame
df_cnrm_cm6_1hr_to_aesa['pr'] = pr

# Salvando dados preditos
df_cnrm_cm6_1hr_to_aesa[columns].to_csv(f'3-INTERPOLACAO/3.2/3.2.3/3.2.3.3/cnrm_cm6_1hr_to_aesa_{database_type}.csv')

# Exibindo as primeiras linhas para verificar
df_cnrm_cm6_1hr_to_aesa[columns].info()
```